In [1]:
import numpy as np
from cellpose import models, core, io
from pathlib import Path
from pathlib import Path
import napari
import apoc
from tqdm import tqdm
import pyclesperanto_prototype as cle 
from tifffile import imwrite, imread
from utils import list_images, read_image

io.logger_setup() # run this to get printing of progress

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose-SAM and Spotiflow models
model = models.CellposeModel(gpu=True)



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 


2025-10-06 14:01:12,469 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2025-10-06 14:01:12,469 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0
2025-10-06 14:01:12,548 [INFO] ** TORCH CUDA version installed and working. **
2025-10-06 14:01:12,549 [INFO] ** TORCH CUDA version installed and working. **
2025-10-06 14:01:12,549 [INFO] >>>> using GPU (CUDA)
2025-10-06 14:01:13,468 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cpsam


In [2]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path(r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\25.09.19\Plate 1 (occludin + claudin-2)")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E2_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E3_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E4_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E5_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E6_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\25.09.19\\Plate 1 (occludin + claudin-2)\\E7_20x.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie

In [6]:
viewer = napari.Viewer(ndisplay=2)

In [3]:
# Explore a different image (0 defines the first image in the directory)
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Extract well_id
well_id = filename.split("_")[0]

# Open one of the multipositions in the img file
img = img[2]

viewer.add_image(img)


Image analyzed: E2_20x


NameError: name 'viewer' is not defined

In [ ]:
img.shape

In [7]:
img = img.transpose(1, 0, 2, 3)
img.shape

(5, 15, 1360, 1360)

In [8]:
# Cellmask and nuclei are in position 2 and 3 respectively

# if you want to combine two stains to create your "cytoplasm" channel
# in this example indices 0, 1 and 2 (1st, 2nd and 3rd) have cellular stains
# and nuclei are in index 3 (4th channel)

img_cp = np.stack((img[[0,1,2]].sum(axis=0), img[3]), axis=0)

img_cp.shape

(2, 15, 1360, 1360)

In [ ]:
viewer.add_image(img_cp)

In [ ]:
labels, flows, styles = model.eval(img_cp, channel_axis=0, z_axis=1, do_3D=True, niter=None) # need to check the arguments
viewer.add_labels(labels)

In [ ]:
maxproj = np.max(img_cp, axis=1)

In [ ]:
viewer.add_image(maxproj)

In [ ]:
maxproj.shape

In [ ]:
labels, flows, styles = model.eval(maxproj[1], do_3D=False, niter=None) # simple nuclei segmentation
viewer.add_labels(labels)

In [ ]:
cle.voronoi_otsu_labeling()

In [4]:
import nd2

def extract_scaling_metadata(filepath):

    with nd2.ND2File(filepath) as nd2_data:
        # Get the first channel's volume metadata
        first_channel = nd2_data.metadata.channels[0]
        voxel_size = first_channel.volume.axesCalibration  # X, Y, Z calibration

        # Extract pixel sizes
        pixel_size_x, pixel_size_y, voxel_size_z = voxel_size

        print(f"Pixel size: {pixel_size_x:.3f} µm x {pixel_size_y:.3f} µm")
        print(f"Voxel (Z-step) size: {voxel_size_z:.3f} µm")

    return pixel_size_x, pixel_size_y, voxel_size_z

In [11]:
img_cp.shape

(2, 15, 1360, 1360)

In [14]:
# Extract x,y,z scaling from .czi file metadata in order to make data isotropic
scaling_x_um, scaling_y_um, scaling_z_um = extract_scaling_metadata(image)

# Adjust so voxel size_x and size_y so they are equal to 1 to avoid compression upon rescaling
multiplier = 1 / scaling_x_um

scaling_x_um = scaling_x_um * multiplier
scaling_y_um = scaling_y_um * multiplier
scaling_z_um = scaling_z_um * multiplier

resampled = cle.scale(img_cp[1], factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)

Pixel size: 0.663 µm x 0.663 µm
Voxel (Z-step) size: 6.631 µm


In [15]:
viewer.add_image(resampled)

<Image layer 'resampled [1]' at 0x2d084c4cd90>